In [82]:
import pandas as pd

Loading the dataframe with abbreviated codes and data ranges. Months were converted to numbers.

In [119]:
jisc_meta = pd.read_csv('jisc-wrangler/data/title_code_lookup.csv', sep='|')

Loading the newspaper overview table, splitting values in the `AcquiredYears` column. Then we create a row for each year using the `.explode()` method.

In [121]:

newspaper_overview = pd.read_csv('jisc-wrangler/data/newspapers_overview.csv',sep=',')
newspaper_overview['AcquiredYears'] = newspaper_overview['AcquiredYears'].str.split()
newspaper_by_year = newspaper_overview.explode('AcquiredYears')


We assume (for now) that each newspapers in the overview spreadsheet runs for the whole year. We add first of January and end of December as start/end years.

In [122]:

newspaper_by_year['Year'] = newspaper_by_year['AcquiredYears']
newspaper_by_year['StartMonth'] = 1
newspaper_by_year['StartDay'] = 1
newspaper_by_year['EndMonth'] = 12
newspaper_by_year['EndDay'] = 12


For each row in the JISC overview table, add a columns that shows the year range between `StartYear` and `EndYear`. The year range is saved as a list in the `Year` column. Subsequently we create a row for each year with `.explode()`.

In [131]:
jisc_meta['Year'] = jisc_meta.apply(
                        lambda x: list(range(x.StartYear,x.EndYear+1)),
                                 axis=1)
jm = jisc_meta.explode('Year')


If the value for `Year` is not the first year the newspaper appeared, use first of January as start the month/day. Same logic is applied to `EndMonth` and `EndDay`.

In [132]:
jm.loc[jm['StartYear'] != jm['Year'],'StartMonth'] = 1
jm.loc[jm['StartYear'] != jm['Year'],'StartDay'] = 1
jm.loc[jm['EndYear'] != jm['Year'],'EndMonth'] = 12
jm.loc[jm['EndYear'] != jm['Year'],'EndDay'] = 31

Add additional columns.

In [133]:
jm['Collection'] = 'JISC'
jm['UK'] = True

In [135]:
# remove columns we won't need in the future
jm.drop(['StartYear','EndYear'],axis=1,inplace=True)
# change NLP to seven numbers code
jm['NLP'] = jm.NLP.apply(lambda x: str(x).zfill(7))

We can enrich the JISC newspapers with information from the BL newspaper titles spreadsheet.

In [130]:
bl_meta = pd.read_excel('jisc-wrangler/data/UK_Ireland_Newspapers_Title_Level_List_v01.xlsx',
                        sheet_name='Title-level list')
# select columns we want to add
bl_subs = bl_meta[['Publication.title','NLP','First.geographical.subject.heading']].drop_duplicates()
# change columns names so they match the newspaper overview dataframe
bl_subs.columns = ['Title','NLP','City']

Merge JISC metadata with the BL catalogue information, using the NLP as join key.

In [136]:
print(jm.shape)
jm_merged = jm.merge(bl_subs,left_on='NLP',right_on='NLP', how='left')
print(jm_merged.shape)

(4635, 9)
(4635, 11)


In [137]:
# this fails for the newspapers?? why?
jm_merged[jm_merged.Title.isnull()].Abbr.unique()

array(['ODFW ', 'RDNP '], dtype=object)

In [ ]:
all_newspapers =pd.concat([newspaper_by_year,jm_merged])
